### WEATHER API
  Instructions:
OpenWeather API
Create an account here at: https://openweathermap.org/ . You will use their API to get weather forecast data
After you have created the free account, request an API key. The key enables you to connect to their API and pull weather data.

  Weather Function
Create a function which will make requests to the OpenWeather Current Weather Data API to get weather information using the latitude and longitude coordinates for a city.
Parse the response from the API request and extract the following information about a city: id,name,country,main description,temperature,minimum temperature,maximum temperature,feels_like,humidity and wind speed.

The function should return a dictionary of the format: {
"id": 802,
"city": "Skopje",
"latitude": 42,
"longtitude": 21.4333,
"country": 25,
"description": "scattered clouds",
"temp":290.97,
"temp_min": 293.97,
"temp_max": 293.97,
"feels_like": 293.23,
"humidity": 43,
"wind_speed": 2.06
}
Design the function in a way that it should work for any given pair of values for latittude and longitute coordinates.
Handle failures in weather function

A failure in this case as an example would be considered passing wrong values for latitude and longtitude. For this reason, modify the weather function you developed before to gracefully fail if wrong values for latitude and longtitude are passed.

The function should capture the response error message from the API call. In these cases, record this information as a new entry into the dataframe . Explore the structure of this dataframe to see what information you need to capture and what columns you need to populate. If required, process the response further to populate the columns accordingly.

In [37]:
import requests

APIkey = '3b44ca87fb86045dc73d251817916e47'
lat = '42.0062'
lon = '20.9698'
r = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={APIkey}')

#print(r.country)
print(r.url)
print(r.status_code)
#print(r.text)
data = r.json()


print(data['city'])
#print(r.json())



https://api.openweathermap.org/data/2.5/forecast?lat=42.0062&lon=20.9698&appid=3b44ca87fb86045dc73d251817916e47
200
{'id': 785082, 'name': 'Tetovo', 'coord': {'lat': 42.0062, 'lon': 20.9698}, 'country': 'MK', 'population': 72944, 'timezone': 3600, 'sunrise': 1762406092, 'sunset': 1762442678}
